In [18]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
train=pd.read_csv("pre_processed_train.csv")
test=pd.read_csv("pre_processed_test.csv")

In [3]:
train

,Unnamed: 0,TransactionID,isFraud,TransactionAmt,card1,card2,card3,card5,addr1,dist1,...,M4_M3,M5_F,M5_N,M5_T,M6_F,M6_N,M6_T,M8_F,M8_N,M8_T
0,0,3424175,0,7.000000,5033.0,269.0,150.0,224.0,330.0,2.449490,...,0,0,0,1,1,0,0,0,1,0
1,1,3206179,0,7.612490,4527.0,376.0,150.0,226.0,441.0,5.986395,...,1,0,1,0,1,0,0,0,1,0
2,2,3410455,0,10.768008,2772.0,512.0,150.0,226.0,143.0,3.741657,...,1,0,1,0,0,0,1,1,0,0
3,3,3058578,0,10.000000,14290.0,512.0,150.0,226.0,177.0,5.986395,...,1,0,1,0,0,1,0,0,1,0
4,4,3242131,0,9.746794,12839.0,321.0,150.0,226.0,220.0,2.449490,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442900,442900,3020019,0,5.385165,3277.0,111.0,150.0,226.0,231.0,5.986395,...,0,0,0,1,1,0,0,0,1,0
442901,442901,3546388,0,9.848858,18132.0,567.0,150.0,117.0,272.0,2.828427,...,0,0,0,1,1,0,0,0,1,0
442902,442902,3087445,0,10.000000,6019.0,583.0,150.0,226.0,441.0,5.986395,...,1,0,1,0,0,1,0,0,1,0
442903,442903,3066953,0,5.000000,10737.0,399.0,150.0,118.0,204.0,5.986395,...,1,0,1,0,0,1,0,0,1,0


In [4]:
t1=train

In [5]:
ytrain=train["isFraud"]
train.drop(axis = 1, labels = ["isFraud","Unnamed: 0"], inplace = True)

In [9]:
test.drop(axis = 1, labels = ["Unnamed: 0"], inplace = True)

In [6]:
over = SMOTE(sampling_strategy=0.04)
under = RandomUnderSampler(sampling_strategy=0.8)
steps = [('o', over),('u',under)]
pipeline = Pipeline(steps=steps)
train1, ytrain1 = pipeline.fit_resample(train, ytrain)

In [7]:
ytrain

0         0
1         0
2         0
3         0
4         0
         ..
442900    0
442901    0
442902    0
442903    0
442904    0
Name: isFraud, Length: 442905, dtype: int64

In [10]:
from sklearn.preprocessing import StandardScaler
scale= StandardScaler()
X_train = scale.fit_transform(train1)
X_test=scale.transform(test) 

In [19]:
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l1','l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
max_iter=[100000]
grid = dict(solver=solvers,penalty=penalty,C=c_values,max_iter=max_iter)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

In [20]:
logreg = LogisticRegression()
grid_search = GridSearchCV(estimator=logreg, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X_train, ytrain1)

/home/darshak/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
300 fits failed out of a total of 900.
The score on these train-test partitions for these parameters will be set to 0.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/home/darshak/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/darshak/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1091, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/home/darshak/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 61, in _check_solver
  

In [21]:
print("Tuned Hyperparameters :", grid_result.best_params_)
print("Accuracy :",grid_result.best_score_)

Tuned Hyperparameters : {'C': 0.1, 'max_iter': 100000, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy : 0.7695454623487772


In [22]:
classifier = LogisticRegression(C=0.1,penalty='l2',solver="lbfgs",max_iter=100000)
classifier.fit(X_train, ytrain1)
y_pred1 = classifier.predict(X_test)

In [23]:
y_pred1.sum()

31044

In [17]:
import csv
with open('log_reg.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id","isFraud"])
    idx=0
    for i in y_pred1:
        writer.writerow([idx,i])
        idx=idx+1